modified based on https://github.com/ekzhu/FLAML/blob/evaluation/evaluation/chess/play_chess.ipynb

In [ ]:
%%capture --no-stderr
%pip install chess -U
%pip install termcolor

In [ ]:
import chess
import chess.svg

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
from flaml import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    # filter_dict={
    #     "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314"],
    # },
)

## Define Agents

In [ ]:
from typing import Dict, List, Optional, Union
from termcolor import colored

sys_msg = """You are an AI-powered chess board agent.
You translate user's natural language input into legal UCI moves.
You should only reply with a UCI move string extracted from user's input."""

user_msg_1 = """Alright, let's get this game started. I'll move pawn from e2 to e4. That's a classic opening, isn't it? Your turn."""

asst_msg_1 = "e2e4"

user_msg_2 = """I am going to move my pawn from e7 to e5."""

asst_msg_2 = "e7e5"

examples = [
    {"role": "user", "content": user_msg_1},
    {"role": "assistant", "content": asst_msg_1},
    {"role": "user", "content": user_msg_2},
    {"role": "assistant", "content": asst_msg_2},
]

class BoardAgent(autogen.AssistantAgent):

    def __init__(self, board: chess.Board):
        super().__init__(
            name="BoardAgent",
            system_message=sys_msg,
            llm_config={"temperature": 0.0, "config_list": config_list},
            max_consecutive_auto_reply=10,
        )
        self._board = board
    
    def generate_reply(
        self,
        messages: Optional[List[Dict]] = None,
        default_reply: Optional[Union[str, Dict]] = "",
        sender: Optional[autogen.Agent] = None,
    ) -> Union[str, Dict, None]:
        # Filter for messages that do not contain error.
        if messages is None:
            messages = self._oai_messages[sender.name]
        filtered_messages = []
        buf = []
        for message in messages:
            buf.append(message)
            if message.get("role") == "assistant":
                if not message.get("content").startswith("Error"):
                    filtered_messages.extend(buf)
                buf = []
        message = messages[-1]
        assert message.get("role") == "user"
        reply = super().generate_reply(messages=filtered_messages + [message], default_reply=default_reply)
        if isinstance(reply, str):
            uci_move = reply
        else:
            uci_move = str(reply["content"])
        try:
            self._board.push_uci(uci_move)
            print(colored(f"uci_move: {uci_move}", "green"))
            m = chess.Move.from_uci(uci_move)
            display(chess.svg.board(self._board, arrows=[(m.from_square, m.to_square)], fill={m.from_square: "gray"}, size=200))
            return uci_move
        except ValueError as e:
            error = f"Error: {e}"
            print(colored(error, "red"))
            return error

In [ ]:
sys_msg_tmpl = """Your name is {name} and you are a chess player. 
You are playing against {opponent_name}. 
You are playing as {color}. 
You communicate your move using universal chess interface language.
You also chit-chat with your opponent when you communicate a move to light up the mood.
You should make sure both you and the opponent are making legal moves."""


class ChessPlayerAgent(autogen.AssistantAgent):

    def __init__(
        self,
        color: str,
        board_agent: BoardAgent,
        max_turns: int,
        **kwargs,
    ):
        if color not in ["white", "black"]:
            raise ValueError(f"color must be either white or black, but got {color}")
        if color == "white":
            opponent_color = "black"
        else:
            opponent_color = "white"
        name = f"Player {color}"
        opponent_name = f"Player {opponent_color}"
        sys_msg = sys_msg_tmpl.format(
            name=name,
            opponent_name=opponent_name,
            color=color,
        )
        super().__init__(
            name=name,
            system_message=sys_msg,
            max_consecutive_auto_reply=max_turns,
            **kwargs,
        )
        self._board_agent = board_agent

    def generate_reply(
        self,
        messages: Optional[List[Dict]] = None,
        default_reply: Optional[Union[str, Dict]] = "",
        sender: Optional[autogen.Agent] = None,
    ) -> Union[str, Dict, None]:
        if messages is None:
            messages = self._oai_messages[sender.name]
        # add a system message about the current state of the board.
        board_state_msg = [{"role": "system", "content": f"Current board:\n{self._board_agent._board}"}]
        if sender == self._board_agent:
            last_message = messages[-1]
            if last_message["content"].startswith("Error"):
                last_message["role"] = "system"
                return super().generate_reply(messages + board_state_msg, default_reply)
            else:
                return
        # TODO: this accesses a private variable of the board agent
        # need to implement a public method for resetting the counter
        self._board_agent._consecutive_auto_reply_counter[self.name] = 0
        self.send(super().generate_reply(messages + board_state_msg, default_reply), self._board_agent)
        # last message sent by the board agent
        last_message = self._oai_messages[self._board_agent.name][-1]
        if last_message["role"] == "assistant":
            # didn't make a legal move after a limit of 10 turns
            print(f"{self.name}: I yield.")
            return
        return self._oai_messages[self._board_agent.name][-2]


## Construct Agents

In [ ]:
board = chess.Board()
board_agent = BoardAgent(board=board)
player_black = ChessPlayerAgent(
    color="black",
    board_agent=board_agent,
    max_turns=30,
    llm_config={"temperature": 0.5, "seed": 1, "config_list": config_list},
)
player_white = ChessPlayerAgent(
    color="white",
    board_agent=board_agent,
    max_turns=30,
    llm_config={"temperature": 0.5, "seed": 2, "config_list": config_list},
)

## Start Game

In [ ]:
player_black.send("Your turn.", player_white)